# Demo on plotting and manipulating grids in Dolfinx

## Basic ```matplotlib``` example

We give a list of vertices and plot them as a grid using ```triplot```.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

pp = np.array([[0.5,1],[2,1],[3,1.5],[3.5,2.5],[2.2,2],[1,2.2]])
ee = np.array([[0,1,5],[1,4,5],[1,2,4],[2,3,4]])
plt.triplot(pp[:,0],pp[:,1],ee)
plt.show()

## ```dolfinx```/```pyvista``` approach

In [ ]:
import dolfinx
import ufl
from mpi4py import MPI
import numpy as np

mesh_comm = MPI.COMM_WORLD
model_rank = 0

import pyvista

def plot_mesh(mesh):
    V0 = dolfinx.fem.functionspace(mesh, ("CG", 1))

    topology, cell_types, x = dolfinx.plot.vtk_mesh(V0)
    grid = pyvista.UnstructuredGrid(topology, cell_types, x)

    plotter = pyvista.Plotter()

    plotter.add_mesh(grid, show_edges=True, show_scalar_bar=True)

    plotter.show()

In [ ]:
gdim = 2
shape = "triangle"
degree = 1

cell = ufl.Cell(shape, geometric_dimension=gdim)
domain = ufl.Mesh(ufl.VectorElement("Lagrange", cell, degree))

x = np.array(pp)
cells = np.array(ee, dtype=np.int32)
mesh = dolfinx.mesh.create_mesh(MPI.COMM_WORLD, cells, x, domain)

# with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "mesh.xdmf", "w") as xdmf:
#     xdmf.write_mesh(mesh)

### 3D grids

First, a test of a very basic single tetroid.

In [ ]:
vertices = np.array([[0., 0., 0.], [1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
cells = np.array([[0, 1, 2, 3]])

gdim = 3
shape = "tetrahedron"
degree = 1

cell = ufl.Cell(shape, geometric_dimension=gdim)
domain = ufl.Mesh(ufl.VectorElement("Lagrange", cell, degree))

mesh = dolfinx.mesh.create_mesh(MPI.COMM_WORLD, cells[:, :4], vertices, domain)

plot_mesh(mesh)

Make it a surface grid

In [ ]:
vertices = np.array([[0., 0., 0.], [1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
cells = np.array([[0, 1, 2], [0, 1, 3], [0, 2, 3]])

gdim = 3
shape = "triangle"
degree = 1

cell = ufl.Cell(shape, geometric_dimension=gdim)
domain = ufl.Mesh(ufl.VectorElement("Lagrange", cell, degree))

mesh = dolfinx.mesh.create_mesh(MPI.COMM_WORLD, cells[:, :4], vertices, domain)

plot_mesh(mesh)

## Grids from file

Read Manuel's grid files and plot the meshes

In [ ]:
vertices = np.loadtxt("Spherical/singleParticle_Tom1_vol.node")
cells = np.loadtxt("Spherical/singleParticle_Tom1_vol.elem", dtype=np.int32)

print(f"# vertices : {len(vertices)}")
print(f"# cell     : {len(cells)}")

In [ ]:
gdim = 3
shape = "tetrahedron"
degree = 1

cell = ufl.Cell(shape, geometric_dimension=gdim)
domain = ufl.Mesh(ufl.VectorElement("Lagrange", cell, degree))

# This approach leads to a kernel crash. :(
mesh = dolfinx.mesh.create_mesh(MPI.COMM_WORLD, cells[:, :4] - 1, vertices, domain)

plot_mesh(mesh)

In [ ]:
V0 = dolfinx.fem.functionspace(mesh, ("CG", 1))

topology, cell_types, x = dolfinx.plot.vtk_mesh(V0)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)

clipped = grid.clip('z')

plotter = pyvista.Plotter()

plotter.add_mesh(clipped, show_edges=True, show_scalar_bar=True)
plotter.add_mesh(grid, style="wireframe", color="blue")

plotter.show()

## The ```GMSH``` approach

In [ ]:
import gmsh
gmsh.initialize()

In [ ]:
lc = 1.e-2  # Some sort of accuracy

for point in vertices:
    gmsh.model.geo.add_point(*point, lc)
    
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate()

# gmsh.fltk.run()

gmsh.finalize()

## The ```meshio``` approach

### Simple mesh

In [ ]:
import meshio
from pathlib import Path
meshio.__version__

In [ ]:
points = np.array([[0.5,1],[2,1],[3,1.5],[3.5,2.5],[2.2,2],[1,2.2]])
cells = [("triangle", np.array([[0,1,5],[1,4,5],[1,2,4],[2,3,4]]))]

io_mesh = meshio.Mesh(points, cells)

io_mesh.write("small_mesh.xdmf")

# write as tetgen
points.tofile("small_mesh.node")
cells[0][1].tofile("small_mesh.ele")

In [ ]:
filename = Path(".", "small_mesh.xdmf")

with dolfinx.io.XDMFFile(MPI.COMM_WORLD, filename, 'r') as file:
    mesh = file.read_mesh(name="Grid") # TODO: name the grid during writing
    
plot_mesh(mesh)

### Cube

In [ ]:
mesh = dolfinx.mesh.create_unit_cube(mesh_comm, 10, 10, 10, cell_type=dolfinx.cpp.mesh.CellType.hexahedron)

topology, cell_types, x = dolfinx.plot.vtk_mesh(mesh, 3)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)

plot_mesh(mesh)

### Sphere

In [ ]:
from dolfinx.io import XDMFFile, gmshio
from pyMoBiMP.gmsh_utils import gmsh_sphere_model, model_to_file

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)

# Create model
model = gmsh.model()

try:
    model = gmsh_sphere_model(model, "Sphere")
    model.setCurrent("Sphere")

    mesh, ct, ft = gmshio.model_to_mesh(model, mesh_comm, rank=0)

    model_to_file(MPI.COMM_SELF, model, "sphere", f"out_gmsh/mesh_rank_{MPI.COMM_WORLD.rank}.xdmf", "w")

finally:
    gmsh.finalize()

In [ ]:
plot_mesh(mesh)

In [ ]:
filename = Path("out_gmsh", "mesh_rank_0.xdmf")

if mesh_comm.rank == model_rank:
    with dolfinx.io.XDMFFile(MPI.COMM_WORLD, filename, 'r') as file:
        mesh = file.read_mesh(name="sphere") # TODO: name the grid during writing

plot_mesh(mesh)

### A simple radial visualization

In [ ]:
V_3d = dolfinx.fem.functionspace(mesh, ("CG", 1))

# Create radial data
mesh_1d = dolfinx.mesh.create_unit_interval(mesh_comm, 10)

V_1d = dolfinx.fem.functionspace(mesh_1d, ("CG", 2))

u_1d = dolfinx.fem.Function(V_1d)
u_1d.interpolate(lambda r: r[0]**2)

import matplotlib.pyplot as plt
import scipy as sp

plt.plot(V_1d.tabulate_dof_coordinates()[:, 0], u_1d.x.array, 'x')


In [ ]:
x = V_1d.tabulate_dof_coordinates()[:, 0]
y = u_1d.x.array

poly = sp.interpolate.lagrange(x, y)

u_3d = dolfinx.fem.Function(V_3d)

u_3d.interpolate(lambda x: poly((x[0]**2 + x[1]**2 + x[2]**2)**0.5))


In [ ]:
topology, cell_types, x = dolfinx.plot.vtk_mesh(V_3d)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)

grid["u"] = u_3d.x.array

clipped = grid.clip_box([0., 1., 0., 1., 0., 1.], crinkle=False)

plotter = pyvista.Plotter()

plotter.add_mesh(clipped, show_edges=True, show_scalar_bar=True)
# plotter.add_mesh(grid, style="wireframe", color="blue")

plotter.show()